In [46]:
import requests
import numpy, pandas
import json
import random
from enum import Enum

In [30]:
class msg_builder:

    headers = {}
    msg_body = {}
    
    def __init__(self, token, message):
        self.headers = { "Authorization": "Bearer " + token }
        self.msg_body = { 'message': message }
        
    def print_content(self):
        print(self.headers)
        print(self.msg_body)

    
    def post_msg(self):
        requests.post("https://notify-api.line.me/api/notify",
    headers = self.headers, data = self.msg_body)

In [55]:
base_url = 'http://openapi.twse.com.tw/v1'

class url_type(Enum):
    Bwibbu_all = '/exchangeReport/BWIBBU_ALL'
    Stock_day_avg_all = '/exchangeReport/STOCK_DAY_AVG_ALL'
    Stock_day_all = '/exchangeReport/STOCK_DAY_ALL'
    Fmsrfk_all = '/exchangeReport/FMSRFK_ALL'
    Fmnptk_all = '/exchangeReport/FMNPTK_ALL'
    Mi_index = '/exchangeReport/MI_INDEX'
    Qfiis_cat = '/fund/MI_QFIIS_cat'
    Qfiis_sort_20 = '/fund/MI_QFIIS_sort_20'
    Twt88u = '/exchangeReport/TWT88U'
    Bfzfzu_t = '/Announcement/BFZFZU_T'
    Twtb4u = '/exchangeReport/TWTB4U'

In [59]:
test_url = base_url + url_type.Stock_day_avg_all.value
test_url

'http://openapi.twse.com.tw/v1/exchangeReport/STOCK_DAY_AVG_ALL'

In [56]:
class data_requester:
    url = ''
    
    def __init__(self, url):
       self.url = url 
    
    def get_url_data(self):
        resp = requests.get(self.url);
        return resp

In [42]:
class data_parser:
    
    def json_converter(self, json_data):
        return json.loads(json_data)

In [33]:
url = 'http://openapi.twse.com.tw/v1/exchangeReport/STOCK_DAY_AVG_ALL'

In [44]:
resp = data_requester(url).get_url_data()
month_data = data_parser().json_converter(resp.text)


In [45]:
month_data

[{'Code': '0050',
  'Name': '元大台灣50',
  'ClosingPrice': '115.30',
  'MonthlyAveragePrice': '117.10'},
 {'Code': '0051',
  'Name': '元大中型100',
  'ClosingPrice': '53.00',
  'MonthlyAveragePrice': '53.33'},
 {'Code': '0052',
  'Name': '富邦科技',
  'ClosingPrice': '98.55',
  'MonthlyAveragePrice': '99.99'},
 {'Code': '0053',
  'Name': '元大電子',
  'ClosingPrice': '55.30',
  'MonthlyAveragePrice': '56.10'},
 {'Code': '0055',
  'Name': '元大MSCI金融',
  'ClosingPrice': '21.81',
  'MonthlyAveragePrice': '22.02'},
 {'Code': '0056',
  'Name': '元大高股息',
  'ClosingPrice': '25.61',
  'MonthlyAveragePrice': '25.76'},
 {'Code': '0057',
  'Name': '富邦摩台',
  'ClosingPrice': '82.55',
  'MonthlyAveragePrice': '83.40'},
 {'Code': '0061',
  'Name': '元大寶滬深',
  'ClosingPrice': '18.65',
  'MonthlyAveragePrice': '18.36'},
 {'Code': '006203',
  'Name': '元大MSCI台灣',
  'ClosingPrice': '56.25',
  'MonthlyAveragePrice': '56.66'},
 {'Code': '006204',
  'Name': '永豐臺灣加權',
  'ClosingPrice': '73.85',
  'MonthlyAveragePrice': '74.70'